# Import librerias

In [1]:
import requests
import json
import pandas as pd
from collections import Counter
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
print("Trabajaremos con las siguientes versiones\n")
print("Pandas",pd.__version__)
print("Numpy",np.__version__)
print("Seaborn", sns.__version__)
!python --version

Trabajaremos con las siguientes versiones

Pandas 1.0.5
Numpy 1.18.5
Seaborn 0.10.1
Python 3.8.3


# Funciones

In [3]:
def import_data(resource_id):

    #Define base url and parameters
    url = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search'
    params = {'resource_id': resource_id, 'limit': 100000000000000} # Change limit size of the retrieved rows: https://docs.ckan.org/en/latest/maintaining/datastore.html#the-data-api  (see parameters in the datastore_search request)

    #Conect to API
    response_API = requests.get(url, params=params)
    if response_API.status_code != 200:
        raise Exception('Could not get data from the API')

    #Get data from API
    data = response_API.json()

    #Get data records
    data_records = data['result']['records']

    #Convert json into a dataframe
    df = pd.json_normalize(data_records)

    return df

In [4]:
# Creamos una función simple para hacer un report simple sobre los DataFrame suministrados
def report_df(dataframe, verbose = True):
    print("El tamaño del dataset es", dataframe.shape[0], "filas y", dataframe.shape[1], "columnas")
    print()
    print(dataframe.info(verbose = verbose))
    total_nulos = dataframe.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos")
    print()
    print("Hay", dataframe.duplicated().sum(), "duplicados")


# CensComercialBCN 2014

## Load data

In [5]:
df_cens = import_data('c933fa10-a965-4509-8de6-e519281dc245').drop(columns = '_id')
df_cens.head()

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
0,"4584527,7",Orange,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,movi,4584527,3644423DF3834D,10,ALTRES,ALTRES,SERVEIS,89,433707,31/03/2014,2012-05-16,Sant Martí,"433707,7",serv
1,"4584524,4",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584524,3644423DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,89,433701,31/03/2014,2012-05-16,Sant Martí,"433701,1",locb
2,4584520,La Llave de oro,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,inmo,4584520,3644426DF3834D,10,ACTIVITATS IMMOBILIÀRIES,SERVEIS,SERVEIS,91,433678,31/03/2014,2012-05-16,Sant Martí,"433678,8",serv
3,"4584515,5",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584515,3644426DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,91,433664,31/03/2014,2012-05-16,Sant Martí,"433664,4",locb
4,"4584511,1",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584511,3644426DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,93,433649,31/03/2014,2012-05-16,Sant Martí,"433649,9",locb


## EDA y preprocesamiento

In [6]:
## Observamos los datos que tenemos 
report_df(df_cens)

df_cens.head(2)

El tamaño del dataset es 67433 filas y 19 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67433 entries, 0 to 67432
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Y_ED50         67433 non-null  object
 1   NOM            67433 non-null  object
 2   CBARRI         67433 non-null  object
 3   NBARRI         67433 non-null  object
 4   CARRCADAST     67433 non-null  object
 5   CODI_EIXOS     67433 non-null  object
 6   Y              67433 non-null  object
 7   RCNUMBER       67433 non-null  object
 8   CDISTRI        67433 non-null  object
 9   BCN3           67433 non-null  object
 10  BCN2           67433 non-null  object
 11  BCN1           67433 non-null  object
 12  DOORNUM        67433 non-null  object
 13  X              67433 non-null  object
 14  ULTIMA_VISITA  67433 non-null  object
 15  DATAINICI      67433 non-null  object
 16  NDISTRIC       67433 non-null  object
 17  X_ED50         67

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
0,"4584527,7",Orange,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,movi,4584527,3644423DF3834D,10,ALTRES,ALTRES,SERVEIS,89,433707,31/03/2014,2012-05-16,Sant Martí,"433707,7",serv
1,"4584524,4",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584524,3644423DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,89,433701,31/03/2014,2012-05-16,Sant Martí,"433701,1",locb


In [7]:
# Examinamos los duplicados existentes
df_cens[df_cens.duplicated()]

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
125,"4583986,6",Local Buit,06,la Sagrada Família,AV DIAGONAL,localb,4583986,1740810DF3814B,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,298,431770,31/03/2014,2013-12-09,Eixample,431770,locb
150,"4583601,1",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,4583601,0835306DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,319,430826,31/03/2014,2013-12-31,Eixample,"430826,6",locb
179,4583750,Local Buit,06,la Sagrada Família,AV DIAGONAL,localb,4583750.37,1037609DF3813G,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,336,431055.96,31/03/2014,2011-07-06,Eixample,"431055,5",locb
208,"4583706,654",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,45837065477,0837728DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,358,43086864201411,31/03/2014,2013-12-31,Eixample,"430868,864",locb
266,"4583499,9",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,4583499.89,0235724DF3803E,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,414,430274.69,31/03/2014,2011-07-06,Eixample,"430274,4",locb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67374,"4588284,4",Varietats,48,la Guineueta,RB CAÃADOR,variet,4588284,0682301DF3808D,08,ALTRES,ALTRES,ALTRES,1,430536,09/03/2014,2014-03-09,Nou Barris,"430536,6",ALTRES
67387,"4582415,511",LLI privada Ninets,08,l Antiga Esquerra de l Eixample,ValÞncia,educ,45824151104439,,02,ENSENYAMENT,SERVEIS,SERVEIS,218-220,42971322259258,16/04/2013,2013-04-16,Eixample,"429713,322",other
67393,"4587906,6",Varietats,48,la Guineueta,RB FABRA I PUIG,variet,4587906,0479903DF3807G,08,ALTRES,ALTRES,ALTRES,474,430454,03/03/2014,2014-03-09,Nou Barris,"430454,4",ALTRES
67400,4587970,Varietats,48,la Guineueta,RB FABRA I PUIG,variet,4587970,0479902DF3807G,08,ALTRES,ALTRES,ALTRES,478,430463,03/03/2014,2014-03-09,Nou Barris,"430463,3",ALTRES


In [8]:
# Comprobamos si estos duplicados, realmente lo son
df_cens[df_cens['RCNUMBER'] == '0837728DF3803F']

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
207,"4583706,654",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,45837065477,0837728DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,358,43086864201411,31/03/2014,2013-12-31,Eixample,"430868,864",locb
208,"4583706,654",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,45837065477,0837728DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,358,43086864201411,31/03/2014,2013-12-31,Eixample,"430868,864",locb


In [9]:
# Decidimos eliminar duplicados, viendo que tiene sentido
df_cens.drop_duplicates(inplace=True)

In [10]:
# Eliminamos columnas que no hacen falta
df_cens =  df_cens.drop(['Y_ED50', 'X_ED50', 'X', 'Y', 
                         'CARRCADAST', 'DOORNUM',
                         'ULTIMA_VISITA', 'DATAINICI',
                         'RCNUMBER'], axis=1)

In [11]:
# Comprobamos los datos que contenemos 
df_cens.describe(exclude=np.number).T

,count,unique,top,freq
NOM,66444,45556,Local Buit,9228
CBARRI,66444,74,07,4142
NBARRI,66444,74,la Dreta de l Eixample,4142
CODI_EIXOS,66444,120,localb,9715
CDISTRI,66444,11,02,14931
BCN3,66444,41,ALTRES,17930
BCN2,66444,9,SERVEIS,30785
BCN1,66444,4,SERVEIS,31244
NDISTRIC,66444,11,Eixample,14931
M_CODI_EIXOS,66444,11,serv,16895


In [12]:
# Probamos los valores que contiene esta columna para entender mejor nuestro dataset
df_cens['NBARRI'].value_counts()

la Dreta de l Eixample             4142
la Vila de Gràcia                  3481
Sant Gervasi - Galvany             2947
l Antiga Esquerra de l Eixample    2810
el Raval                           2540
                                   ... 
#N/A                                 21
Baró de Viver                        21
Torre Baró                           20
Vallbona                              8
la Clota                              6
Name: NBARRI, Length: 74, dtype: int64

In [13]:
# Remplazamos los valores nulos para que se puedan detectar
df_cens.replace('#N/A',np.nan,inplace=True)

In [14]:
# Comprobamos que se hizo el cambio correctamente
df_cens[df_cens.isnull().any(axis=1)].head(2)

,NOM,CBARRI,NBARRI,CODI_EIXOS,CDISTRI,BCN3,BCN2,BCN1,NDISTRIC,M_CODI_EIXOS
3556,CLUB DE PETANCA CASA OLIVA,NaN,NaN,espo,NaN,ALTRES,ALTRES,ALTRES,NaN,ALTRES
14999,Institut Lluís Vives,NaN,NaN,educ,NaN,ENSENYAMENT,SERVEIS,SERVEIS,NaN,other


In [15]:
# Eliminamos valores nulos
df_cens.dropna(inplace=True)
df_cens.reset_index(drop=True,inplace=True)

In [16]:
# Renombramos columnas para trabajar más fácilmente
df_cens.rename(columns={'CBARRI': 'Codi_Barri', 'NBARRI': 'Nom_Barri', 'CDISTRI': 'Codi_Districte', 'NDISTRIC': 'Nom_Districte'}, inplace=True)

In [17]:
df_cens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66423 entries, 0 to 66422
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NOM             66423 non-null  object
 1   Codi_Barri      66423 non-null  object
 2   Nom_Barri       66423 non-null  object
 3   CODI_EIXOS      66423 non-null  object
 4   Codi_Districte  66423 non-null  object
 5   BCN3            66423 non-null  object
 6   BCN2            66423 non-null  object
 7   BCN1            66423 non-null  object
 8   Nom_Districte   66423 non-null  object
 9   M_CODI_EIXOS    66423 non-null  object
dtypes: object(10)
memory usage: 5.1+ MB


In [18]:
# Comprobamos diferencias entre columnas BCN3, BCN2 y BCN1
df_cens[(df_cens['NOM']=='Local Buit') & (df_cens['BCN2']!= 'LOCAL BUIT')]

,NOM,Codi_Barri,Nom_Barri,CODI_EIXOS,Codi_Districte,BCN3,BCN2,BCN1,Nom_Districte,M_CODI_EIXOS
2912,Local Buit,27,el Putxet i el Farró,moda,05,VESTIR,EQUIPAMENT PERSONAL,COMERÇ AL DETALL,Sarrià-Sant Gervasi,pers
3833,Local Buit,19,les Corts,ofic,04,ALTRES,SERVEIS,SERVEIS,Les Corts,serv
9869,Local Buit,25,Sant Gervasi - la Bonanova,moda,05,VESTIR,EQUIPAMENT PERSONAL,COMERÇ AL DETALL,Sarrià-Sant Gervasi,pers
11022,Local Buit,26,Sant Gervasi - Galvany,moda,05,VESTIR,EQUIPAMENT PERSONAL,COMERÇ AL DETALL,Sarrià-Sant Gervasi,pers
12407,Local Buit,17,Sants - Badal,colma,03,ALIMENTACIÓ,QUOTIDIÀ ALIMENTARI,COMERÇ AL DETALL,Sants-Montjuïc,quot
12413,Local Buit,46,el Turó de la Peira,locu,08,ALTRES,SERVEIS,SERVEIS,Nou Barris,serv
15234,Local Buit,60,Sant Andreu,tall,09,REPARACIONS (ELECTRODOMÈSTICS I AUTOMÒBILS),SERVEIS,SERVEIS,Sant Andreu,tran
16017,Local Buit,18,Sants,inmo,03,ACTIVITATS IMMOBILIÀRIES,SERVEIS,SERVEIS,Sants-Montjuïc,serv
18470,Local Buit,44,Vilapicina i la Torre Llobeta,ofic,08,ALTRES,SERVEIS,SERVEIS,Nou Barris,serv
19589,Local Buit,18,Sants,arranj,03,"MANTENIMENT, NETEJA I PRODUCCIÓ",SERVEIS,SERVEIS,Sants-Montjuïc,serv


In [19]:
# Entender mejor los datos que contiene, vemos que se parece a las columnas de BCN3,2 y 1
df_cens['M_CODI_EIXOS'].value_counts()

serv      16895
host      10249
locb       9715
pers       6835
quot       6104
ALTRES     4069
altr       2958
llar       2785
tran       2522
cult       2499
other      1792
Name: M_CODI_EIXOS, dtype: int64

In [20]:
df_cens['BCN1'].value_counts()

SERVEIS             31239
COMERÇ AL DETALL    21400
LOCAL BUIT           9715
ALTRES               4069
Name: BCN1, dtype: int64

In [21]:
df_cens['BCN3'].value_counts()

ALTRES                                                            17914
RESTAURANTS, BARS I HOTELS (INCLÒS HOSTALS, PENSIONS I FONDES)    10249
LOCAL BUIT                                                         9715
VESTIR                                                             3488
ALIMENTACIÓ                                                        2406
ENSENYAMENT                                                        2239
PA, PASTISSERIA I LÀCTICS                                          1705
MOBLES I ARTICLES FUSTA I METALL                                   1639
FINANCES I ASSEGURANCES                                            1551
SANITAT I ASSISTÈNCIA                                              1385
REPARACIONS (ELECTRODOMÈSTICS I AUTOMÒBILS)                        1283
LLIBRES, DIARIS I REVISTES                                         1125
FARMÀCIES                                                          1065
FRUITES I VERDURES                                              

In [22]:
df_cens['BCN2'].value_counts()

SERVEIS                   30780
LOCAL BUIT                 9715
QUOTIDIÀ ALIMENTARI        6569
ALTRES                     6277
EQUIPAMENT PERSONAL        5114
PARAMENT DE LA LLAR        3086
OCI I CULTURA              2798
QUOTIDIÀ NO ALIMENTARI     1724
AUTOMOCIÓ                   360
Name: BCN2, dtype: int64

In [23]:
df_cens.columns

Index(['NOM', 'Codi_Barri', 'Nom_Barri', 'CODI_EIXOS', 'Codi_Districte',
       'BCN3', 'BCN2', 'BCN1', 'Nom_Districte', 'M_CODI_EIXOS'],
      dtype='object')

In [24]:
# Eliminamos más columnas que vimos que contiene valores parecidos para reducir nuestro dataset de trabajo
df_cens.drop(['NOM','CODI_EIXOS','BCN3','BCN1','M_CODI_EIXOS'],axis=1,inplace=True)

In [25]:
df_cens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66423 entries, 0 to 66422
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Codi_Barri      66423 non-null  object
 1   Nom_Barri       66423 non-null  object
 2   Codi_Districte  66423 non-null  object
 3   BCN2            66423 non-null  object
 4   Nom_Districte   66423 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB


In [26]:
# Cambio nombre columna por un nombre más adecuado al contenido
df_cens.rename(columns = {'BCN2':'Activitat_Economica'},inplace=True)

In [27]:
df_cens.head()

,Codi_Barri,Nom_Barri,Codi_Districte,Activitat_Economica,Nom_Districte
0,69,Diagonal Mar i el Front Marítim del Poblenou,10,ALTRES,Sant Martí
1,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí
2,69,Diagonal Mar i el Front Marítim del Poblenou,10,SERVEIS,Sant Martí
3,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí
4,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí


In [28]:
report_df(df_cens)

El tamaño del dataset es 66423 filas y 5 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66423 entries, 0 to 66422
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Codi_Barri           66423 non-null  object
 1   Nom_Barri            66423 non-null  object
 2   Codi_Districte       66423 non-null  object
 3   Activitat_Economica  66423 non-null  object
 4   Nom_Districte        66423 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB
None

Tenemos un total de 0 nulos

Hay 65818 duplicados


## Cálculo de actividad por barrio

In [29]:
# Groupby para calcular el total de actividades que hay según el barrio y el tipo de actividad economica
Activitat_Barris = df_cens.groupby(['Codi_Barri','Activitat_Economica']).size().to_frame(name = 'Total_Activitat_Per_Barri').reset_index()

In [30]:
# Imprimimos el dataframe generado para ver como queda y ver si tiene  sentido
Activitat_Barris

,Codi_Barri,Activitat_Economica,Total_Activitat_Per_Barri
0,01,ALTRES,381
1,01,AUTOMOCIÓ,1
2,01,EQUIPAMENT PERSONAL,225
3,01,LOCAL BUIT,245
4,01,OCI I CULTURA,88
...,...,...,...
600,73,OCI I CULTURA,26
601,73,PARAMENT DE LA LLAR,43
602,73,QUOTIDIÀ ALIMENTARI,116
603,73,QUOTIDIÀ NO ALIMENTARI,28


In [31]:
# Comprobación visual de un barrio aleatorio
Activitat_Barris[Activitat_Barris['Codi_Barri']=='69']

,Codi_Barri,Activitat_Economica,Total_Activitat_Per_Barri
561,69,ALTRES,20
562,69,EQUIPAMENT PERSONAL,8
563,69,LOCAL BUIT,42
564,69,OCI I CULTURA,17
565,69,PARAMENT DE LA LLAR,3
566,69,QUOTIDIÀ ALIMENTARI,19
567,69,QUOTIDIÀ NO ALIMENTARI,11
568,69,SERVEIS,185


In [32]:
# merge del dataframe original con el dataframe generado con el groupby anterior. Comprobamos que no se haga un cartesiano
print(df_cens.shape)

df_cens = pd.merge(df_cens,Activitat_Barris, on = ['Codi_Barri','Activitat_Economica'], how = 'left')

print(df_cens.shape)

(66423, 5)
(66423, 6)


In [33]:
# imprimimos por pantalla para comprobar como queda
df_cens.head()

,Codi_Barri,Nom_Barri,Codi_Districte,Activitat_Economica,Nom_Districte,Total_Activitat_Per_Barri
0,69,Diagonal Mar i el Front Marítim del Poblenou,10,ALTRES,Sant Martí,20
1,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí,42
2,69,Diagonal Mar i el Front Marítim del Poblenou,10,SERVEIS,Sant Martí,185
3,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí,42
4,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí,42


In [34]:
# pivot table para que las actividades economicas se coniertan en columnas y reducir un poco la dimensionalidad
df_cens_pivot = pd.pivot_table(df_cens, values='Total_Activitat_Per_Barri', index=['Codi_Districte','Nom_Districte','Codi_Barri','Nom_Barri'], columns='Activitat_Economica', fill_value=0)

In [35]:
df_cens_pivot.reset_index(inplace=True)

In [36]:
# cambio de nombre de columnas para estar alineados con el equipo de Back
df_cens_pivot.rename(columns = {
    'Codi_Districte':'codiDistricte',
    'Nom_Districte' : 'nomDistricte',
    'Nom_Barri':'nomBarri',
    'Codi_Barri':'codiBarri',
    'Nom_Barri':'nomBarri',
    'ALTRES':'nAltres',
    'AUTOMOCIÓ':'nAutomocio',
    'EQUIPAMENT PERSONAL':'nEquipamentPersonal',
    'LOCAL BUIT' : 'nLocalBuit',
    'OCI I CULTURA': 'nOciCultura',
    'PARAMENT DE LA LLAR':'nParamentLlar',
    'QUOTIDIÀ ALIMENTARI' : 'nQuotidiaAlimentari',
    'QUOTIDIÀ NO ALIMENTARI' : 'nQuotidiaNoAlimentari',
    'SERVEIS':'nServeis'
    
},inplace=True

)

In [37]:
# generamos una nueva columna con el año del dataset para más tarde añadir otro años
df_cens_pivot['Anio'] = '2014'

In [38]:
df_cens_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   codiDistricte          73 non-null     object
 1   nomDistricte           73 non-null     object
 2   codiBarri              73 non-null     object
 3   nomBarri               73 non-null     object
 4   nAltres                73 non-null     int64 
 5   nAutomocio             73 non-null     int64 
 6   nEquipamentPersonal    73 non-null     int64 
 7   nLocalBuit             73 non-null     int64 
 8   nOciCultura            73 non-null     int64 
 9   nParamentLlar          73 non-null     int64 
 10  nQuotidiaAlimentari    73 non-null     int64 
 11  nQuotidiaNoAlimentari  73 non-null     int64 
 12  nServeis               73 non-null     int64 
 13  Anio                   73 non-null     object
dtypes: int64(9), object(5)
memory usage: 8.1+ KB


In [39]:
# Convertimos el año en formato fecha
df_cens_pivot['Anio'] = pd.to_datetime(df_cens_pivot['Anio'],format = "%Y")

In [40]:
df_cens_pivot

Activitat_Economica,codiDistricte,nomDistricte,codiBarri,nomBarri,nAltres,nAutomocio,nEquipamentPersonal,nLocalBuit,nOciCultura,nParamentLlar,nQuotidiaAlimentari,nQuotidiaNoAlimentari,nServeis,Anio
0,01,Ciutat Vella,01,el Raval,381,1,225,245,88,137,296,47,1120,2014-01-01
1,01,Ciutat Vella,02,el Barri Gòtic,315,0,478,199,78,76,161,37,786,2014-01-01
2,01,Ciutat Vella,03,la Barceloneta,99,0,26,108,24,13,89,16,370,2014-01-01
3,01,Ciutat Vella,04,"Sant Pere, Santa Caterina i la Ribera",259,1,363,155,53,98,175,34,688,2014-01-01
4,02,Eixample,05,el Fort Pienc,81,12,40,199,47,55,113,33,556,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,20,0,8,42,17,3,19,11,185,2014-01-01
69,10,Sant Martí,70,el Besòs i el Maresme,90,1,16,59,12,22,71,18,204,2014-01-01
70,10,Sant Martí,71,Provençals del Poblenou,166,5,11,122,26,16,40,19,314,2014-01-01
71,10,Sant Martí,72,Sant Martí de Provençals,80,4,40,120,23,34,84,18,336,2014-01-01


# CensComercialBCN 2016

## Load data

In [41]:
# Import data from API
df_cens_2016 = import_data('18cbddda-ad3f-41b8-8229-064f0faa930b').drop(columns = '_id')
df_cens_2016.head()

,NUM_POLICI,N_EIX,SN_CCOMERC,Codi_Districte,ID_CCOMERC,N_PRINCIP,DATA,LATITUD,SN_GALERIA,ID_BCN,...,LONGITUD,N_SECTOR,REF_CAD,N_GRUPACT,Nom_Barri,N_CARRER,N_GALERIA,N_CCOMERC,ID_SECTOR,ID_GRUPACT
0,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.375377,0,49,...,2.182944,Comerç al detall,1813205DF3811D,Equipament personal,el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,1,4
1,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.37539,0,50,...,2.183092,Serveis,1813205DF3811D,"Restaurants, bars i hotels (Inclòs hostals, pe...",el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,2,14
2,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.375404,0,51,...,2.183217,Comerç al detall,1813205DF3811D,Equipament personal,el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,1,4
3,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.37542,0,52,...,2.183354,Comerç al detall,1813205DF3811D,Equipament personal,el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,1,4
4,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.375425,0,53,...,2.183517,Serveis,1813205DF3811D,"Restaurants, bars i hotels (Inclòs hostals, pe...",el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,2,14


## EDA y preprocesamiento

In [42]:
report_df(df_cens_2016)

El tamaño del dataset es 78033 filas y 34 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78033 entries, 0 to 78032
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NUM_POLICI      78017 non-null  object
 1   N_EIX           78033 non-null  object
 2   SN_CCOMERC      78033 non-null  object
 3   Codi_Districte  78033 non-null  object
 4   ID_CCOMERC      78033 non-null  object
 5   N_PRINCIP       78033 non-null  object
 6   DATA            78033 non-null  object
 7   LATITUD         78033 non-null  object
 8   SN_GALERIA      78033 non-null  object
 9   ID_BCN          78033 non-null  object
 10  N_MERCAT        78033 non-null  object
 11  N_LOCAL         78033 non-null  object
 12  ID_PRINCIP      78033 non-null  object
 13  SN_CARRER       78033 non-null  object
 14  SEC_CENS        78033 non-null  object
 15  SN_EIX          78033 non-null  object
 16  Y_UTM_ETRS      78033 non-null  object
 17

In [43]:
# Eliminar columnas innecesarias
df_cens_2016 =  df_cens_2016.drop(['NUM_POLICI', 'N_EIX', 'SN_CCOMERC', 'ID_CCOMERC',
                       'N_PRINCIP', 'DATA', 'LATITUD', 'SN_GALERIA', 'ID_BCN', 'N_MERCAT',
                       'N_LOCAL', 'ID_PRINCIP', 'SN_CARRER', 'SEC_CENS', 'SN_EIX',
                       'Y_UTM_ETRS', 'X_UTM_ETRS', 'ID_ACT', 'ID_MERCAT', 
                       'SN_MERCAT', 'LONGITUD', 'REF_CAD',
                       'N_CARRER', 'N_GALERIA', 'N_CCOMERC',
                       'ID_SECTOR', 'ID_GRUPACT'], axis=1)

In [44]:
# Cambiamos el nombre de la columna para que tenga más coherencia con los demás datos con los que trabajamosç
df_cens_2016.rename(columns={'N_DISTRI': 'Nom_Districte'}, inplace=True)

In [45]:
df_cens_2016.describe(exclude=np.number).T

,count,unique,top,freq
Codi_Districte,78033,10,2,16179
Nom_Districte,78033,10,Eixample,16179
Codi_Barri,78033,73,31,4581
N_ACT,78033,62,-,17768
N_SECTOR,78033,8,Serveis,31773
N_GRUPACT,78033,16,Altres,20867
Nom_Barri,78033,75,la Vila de Gràcia,4581


Después de analizar los valores de las columnas N_DISTRI, N_SECTOR, N_GRUPACT (que contienen información sobre los tipos de actividad economica) nos decantamos por N_GRUPACT.

In [46]:
print("Número de valores únicos: " + str(len(df_cens_2016['N_ACT'].unique())))
df_cens_2016['N_ACT'].value_counts()

Número de valores únicos: 62


-                                 17768
Serveis de menjar i begudes        9680
Vestir                             3296
Resta alimentació                  3266
Altres                             2906
                                  ...  
Música                               84
Combustibles i carburants            75
Fabricació tèxtil                    75
Maquinària                           60
Grans magatzems i hipermercats       30
Name: N_ACT, Length: 62, dtype: int64

In [47]:
print("Número de valores únicos: " + str(len(df_cens_2016['N_SECTOR'].unique())))
df_cens_2016['N_SECTOR'].value_counts()

Número de valores únicos: 8


Serveis                            31773
Comerç al detall                   22747
Sense informació                   13023
Altres                              5745
Locals buits en lloguer             2208
Locals buits en venda i lloguer     1052
Locals buits en venda                787
En reforma                           698
Name: N_SECTOR, dtype: int64

In [48]:
print("Número de valores únicos: " + str(len(df_cens_2016['N_GRUPACT'].unique())))
df_cens_2016['N_GRUPACT'].value_counts()

Número de valores únicos: 16


Altres                                                            20867
-                                                                 17768
Restaurants, bars i hotels (Inclòs hostals, pensions i fondes)    10339
Quotidià alimentari                                                8536
Equipament personal                                                5175
Quotidià no alimentari                                             2181
Parament de la llar                                                2101
Ensenyament                                                        1907
Oci i cultura                                                      1801
Sanitat i assistència                                              1698
Reparacions (Electrodomèstics i automòbils)                        1511
Finances i assegurances                                            1511
Equipaments culturals i recreatius                                 1236
Activitats immobiliàries                                        

In [49]:
df_cens_2016 =  df_cens_2016.drop(['N_ACT', 'N_SECTOR'], axis=1)
df_cens_2016.head(3)

,Codi_Districte,Nom_Districte,Codi_Barri,N_GRUPACT,Nom_Barri
0,1,Ciutat Vella,2,Equipament personal,el Barri Gòtic
1,1,Ciutat Vella,2,"Restaurants, bars i hotels (Inclòs hostals, pe...",el Barri Gòtic
2,1,Ciutat Vella,2,Equipament personal,el Barri Gòtic


Como vemos, N_GRUPACT no tiene exactamente los mismos valores que la clasificación usada en los datos de 2014, por lo tanto cambiamos los valores. Como hemos podido comprobar inspecionando manualmente el anterior dataset, todos los valores nuevos estan classificados como "serveis" en el dataset de 2014. Por lo tanto, los sustituimos.

Por otro lado, también decidimos cambiar los valores faltantes (representados como '-'), por 'Altres'. Hemos tomado esta decisión para evitar perder datos, ya que estamos hablando de 17768 filas, pero evidentemente otra opción hubiera sido eliminarlos.

In [50]:
df_cens_2016['N_GRUPACT'] = df_cens_2016['N_GRUPACT'].replace({'Restaurants, bars i hotels (Inclòs hostals, pensions i fondes)': 'SERVEIS',
                                                    'Ensenyament': 'SERVEIS',
                                                    'Reparacions (Electrodomèstics i automòbils)': 'SERVEIS',
                                                    'Sanitat i assistència': 'SERVEIS',
                                                    'Finances i assegurances': 'SERVEIS',
                                                    'Equipaments culturals i recreatius': 'SERVEIS',
                                                    'Manteniment, neteja i producció': 'SERVEIS',
                                                    'Activitats immobiliàries': 'SERVEIS',
                                                    '-': 'Altres'})

df_cens_2016['N_GRUPACT'].value_counts()

Altres                    38635
SERVEIS                   19331
Quotidià alimentari        8536
Equipament personal        5175
Quotidià no alimentari     2181
Parament de la llar        2101
Oci i cultura              1801
Automoció                   273
Name: N_GRUPACT, dtype: int64

Hemos detectado que hay algun error en los barrios ya que salen 75. Después de investigarlo hemos llegado a la conclusión de hay barrios repetidos pero con diferencias en mayúsculas y minúsculas, así que pasamos todo a minúsculas.

In [51]:
df_cens_2016.describe(exclude=np.number).T

,count,unique,top,freq
Codi_Districte,78033,10,2,16179
Nom_Districte,78033,10,Eixample,16179
Codi_Barri,78033,73,31,4581
N_GRUPACT,78033,8,Altres,38635
Nom_Barri,78033,75,la Vila de Gràcia,4581


In [52]:
df_cens_2016['Nom_Barri'] = df_cens_2016['Nom_Barri'].str.lower()
len(df_cens_2016['Nom_Barri'].value_counts())

73

In [53]:
df_cens_2016

,Codi_Districte,Nom_Districte,Codi_Barri,N_GRUPACT,Nom_Barri
0,1,Ciutat Vella,2,Equipament personal,el barri gòtic
1,1,Ciutat Vella,2,SERVEIS,el barri gòtic
2,1,Ciutat Vella,2,Equipament personal,el barri gòtic
3,1,Ciutat Vella,2,Equipament personal,el barri gòtic
4,1,Ciutat Vella,2,SERVEIS,el barri gòtic
...,...,...,...,...,...
78028,5,Sarrià-Sant Gervasi,26,Equipament personal,sant gervasi galvany
78029,5,Sarrià-Sant Gervasi,26,Parament de la llar,sant gervasi galvany
78030,5,Sarrià-Sant Gervasi,26,Altres,sant gervasi galvany
78031,5,Sarrià-Sant Gervasi,26,SERVEIS,sant gervasi galvany


In [54]:
# Cambio nombre columna por un nombre más adecuado al contenido
df_cens_2016.rename(columns = {'N_GRUPACT':'Activitat_Economica'},inplace=True)

## Cálculo de actividad por barrio

In [55]:
# Groupby para calcular el total de actividades que hay según el barrio y el tipo de actividad economica
Activitat_Barris_2016 = df_cens_2016.groupby(['Codi_Barri','Activitat_Economica']).size().to_frame(name = 'Total_Activitat_Per_Barri').reset_index()
Activitat_Barris_2016

,Codi_Barri,Activitat_Economica,Total_Activitat_Per_Barri
0,1,Altres,1316
1,1,Automoció,1
2,1,Equipament personal,208
3,1,Oci i cultura,124
4,1,Parament de la llar,88
...,...,...,...
530,9,Oci i cultura,77
531,9,Parament de la llar,74
532,9,Quotidià alimentari,236
533,9,Quotidià no alimentari,70


In [56]:
# merge del dataframe original con el dataframe generado con el groupby anterior. Comprobamos que no se haga un cartesiano
print(df_cens_2016.shape)

df_cens_2016 = pd.merge(df_cens_2016,Activitat_Barris_2016, on = ['Codi_Barri','Activitat_Economica'], how = 'left')

print(df_cens_2016.shape)

(78033, 5)
(78033, 6)


In [57]:
# pivot table para que las actividades economicas se coniertan en columnas y reducir un poco la dimensionalidad
df_cens_pivot_2016 = pd.pivot_table(df_cens_2016, values='Total_Activitat_Per_Barri', index=['Codi_Districte','Nom_Districte','Codi_Barri','Nom_Barri'], columns='Activitat_Economica', fill_value=0)
df_cens_pivot_2016.reset_index(inplace=True)

In [58]:
df_cens_pivot_2016

Activitat_Economica,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Altres,Automoció,Equipament personal,Oci i cultura,Parament de la llar,Quotidià alimentari,Quotidià no alimentari,SERVEIS
0,1,Ciutat Vella,1,el raval,1316,1,208,124,88,574,58,660
1,1,Ciutat Vella,2,el barri gòtic,817,0,484,52,49,151,43,560
2,1,Ciutat Vella,3,la barceloneta,256,0,19,12,7,110,16,248
3,1,Ciutat Vella,4,"sant pere, santa caterina i la ribera",812,1,290,35,66,254,40,464
4,10,Sant Martí,64,el camp de l'arpa del clot,946,9,85,36,55,175,52,376
...,...,...,...,...,...,...,...,...,...,...,...,...
68,9,Sant Andreu,59,el bon pastor,474,20,104,16,23,52,32,158
69,9,Sant Andreu,60,sant andreu,1180,4,124,52,67,237,69,533
70,9,Sant Andreu,61,la sagrera,465,3,34,16,21,131,30,213
71,9,Sant Andreu,62,el congrés i els indians,364,1,22,9,17,52,20,180


In [59]:
# cambio de nombre de columnas para estar alineados con el equipo de Back
df_cens_pivot_2016.rename(columns = {
    'Codi_Districte':'codiDistricte',
    'Nom_Districte' : 'nomDistricte',
    'Nom_Barri':'nomBarri',
    'Codi_Barri':'codiBarri',
    'Nom_Barri':'nomBarri',
    'Altres':'nAltres',
    'Automoció':'nAutomocio',
    'Equipament personal':'nEquipamentPersonal',
    'Oci i cultura': 'nOciCultura',
    'Parament de la llar':'nParamentLlar',
    'Quotidià alimentari' : 'nQuotidiaAlimentari',
    'Quotidià no alimentari' : 'nQuotidiaNoAlimentari',
    'SERVEIS':'nServeis'
    
},inplace=True

)

In [60]:
# generamos una nueva columna con el año del dataset
df_cens_pivot_2016['Anio'] = '2016'

In [61]:
df_cens_pivot_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   codiDistricte          73 non-null     object
 1   nomDistricte           73 non-null     object
 2   codiBarri              73 non-null     object
 3   nomBarri               73 non-null     object
 4   nAltres                73 non-null     int64 
 5   nAutomocio             73 non-null     int64 
 6   nEquipamentPersonal    73 non-null     int64 
 7   nOciCultura            73 non-null     int64 
 8   nParamentLlar          73 non-null     int64 
 9   nQuotidiaAlimentari    73 non-null     int64 
 10  nQuotidiaNoAlimentari  73 non-null     int64 
 11  nServeis               73 non-null     int64 
 12  Anio                   73 non-null     object
dtypes: int64(8), object(5)
memory usage: 7.5+ KB


In [62]:
# Convertimos el año en formato fecha
df_cens_pivot_2016['Anio'] = pd.to_datetime(df_cens_pivot_2016['Anio'],format = "%Y")

In [63]:
df_cens_pivot_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   codiDistricte          73 non-null     object        
 1   nomDistricte           73 non-null     object        
 2   codiBarri              73 non-null     object        
 3   nomBarri               73 non-null     object        
 4   nAltres                73 non-null     int64         
 5   nAutomocio             73 non-null     int64         
 6   nEquipamentPersonal    73 non-null     int64         
 7   nOciCultura            73 non-null     int64         
 8   nParamentLlar          73 non-null     int64         
 9   nQuotidiaAlimentari    73 non-null     int64         
 10  nQuotidiaNoAlimentari  73 non-null     int64         
 11  nServeis               73 non-null     int64         
 12  Anio                   73 non-null     datetime64[ns]
dtypes: date

In [64]:
df_cens_pivot_2016

Activitat_Economica,codiDistricte,nomDistricte,codiBarri,nomBarri,nAltres,nAutomocio,nEquipamentPersonal,nOciCultura,nParamentLlar,nQuotidiaAlimentari,nQuotidiaNoAlimentari,nServeis,Anio
0,1,Ciutat Vella,1,el raval,1316,1,208,124,88,574,58,660,2016-01-01
1,1,Ciutat Vella,2,el barri gòtic,817,0,484,52,49,151,43,560,2016-01-01
2,1,Ciutat Vella,3,la barceloneta,256,0,19,12,7,110,16,248,2016-01-01
3,1,Ciutat Vella,4,"sant pere, santa caterina i la ribera",812,1,290,35,66,254,40,464,2016-01-01
4,10,Sant Martí,64,el camp de l'arpa del clot,946,9,85,36,55,175,52,376,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,9,Sant Andreu,59,el bon pastor,474,20,104,16,23,52,32,158,2016-01-01
69,9,Sant Andreu,60,sant andreu,1180,4,124,52,67,237,69,533,2016-01-01
70,9,Sant Andreu,61,la sagrera,465,3,34,16,21,131,30,213,2016-01-01
71,9,Sant Andreu,62,el congrés i els indians,364,1,22,9,17,52,20,180,2016-01-01


## Concatenar datasets

Concatenamos con el dataset anterior para tener la información sobre los dos años.

In [65]:
# Concatenamos con el anterior dataset
df_cens_concat = pd.concat([df_cens_pivot, df_cens_pivot_2016], ignore_index=True)
df_cens_concat

,codiDistricte,nomDistricte,codiBarri,nomBarri,nAltres,nAutomocio,nEquipamentPersonal,nLocalBuit,nOciCultura,nParamentLlar,nQuotidiaAlimentari,nQuotidiaNoAlimentari,nServeis,Anio
0,01,Ciutat Vella,01,el Raval,381,1,225,245.0,88,137,296,47,1120,2014-01-01
1,01,Ciutat Vella,02,el Barri Gòtic,315,0,478,199.0,78,76,161,37,786,2014-01-01
2,01,Ciutat Vella,03,la Barceloneta,99,0,26,108.0,24,13,89,16,370,2014-01-01
3,01,Ciutat Vella,04,"Sant Pere, Santa Caterina i la Ribera",259,1,363,155.0,53,98,175,34,688,2014-01-01
4,02,Eixample,05,el Fort Pienc,81,12,40,199.0,47,55,113,33,556,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,9,Sant Andreu,59,el bon pastor,474,20,104,NaN,16,23,52,32,158,2016-01-01
142,9,Sant Andreu,60,sant andreu,1180,4,124,NaN,52,67,237,69,533,2016-01-01
143,9,Sant Andreu,61,la sagrera,465,3,34,NaN,16,21,131,30,213,2016-01-01
144,9,Sant Andreu,62,el congrés i els indians,364,1,22,NaN,9,17,52,20,180,2016-01-01


Como vemos, en este caso faltaria rellenar los datos de 'nLocalBuit' para el segundo dataset ya que no estaban presentes en la columna escogida para la actividad económica. Sería el siguiente paso a hacer. También se deberia tener en cuenta la diferencia en los nombres de les barrios.

# CensComercialBCN 2019

In [70]:
df_cens_2019 = import_data('18cbddda-ad3f-41b8-8229-064f0faa930b').drop(columns = '_id')
df_cens_2019.head()

,NUM_POLICI,N_EIX,SN_CCOMERC,Codi_Districte,ID_CCOMERC,N_PRINCIP,DATA,LATITUD,SN_GALERIA,ID_BCN,...,LONGITUD,N_SECTOR,REF_CAD,N_GRUPACT,Nom_Barri,N_CARRER,N_GALERIA,N_CCOMERC,ID_SECTOR,ID_GRUPACT
0,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.375377,0,49,...,2.182944,Comerç al detall,1813205DF3811D,Equipament personal,el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,1,4
1,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.37539,0,50,...,2.183092,Serveis,1813205DF3811D,"Restaurants, bars i hotels (Inclòs hostals, pe...",el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,2,14
2,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.375404,0,51,...,2.183217,Comerç al detall,1813205DF3811D,Equipament personal,el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,1,4
3,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.37542,0,52,...,2.183354,Comerç al detall,1813205DF3811D,Equipament personal,el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,1,4
4,5,,1,1,CC003,Actiu,2016-06-07T00:00:00,41.375425,0,53,...,2.183517,Serveis,1813205DF3811D,"Restaurants, bars i hotels (Inclòs hostals, pe...",el Barri Gòtic,MOLL ESPANYA,,Maremàgnum,2,14


Para los datos del 2019 seguiriamos el mismo procedimiento que como hasta ahora, seleccionando las columnas de barrio, distrito y actividad economica correspondientes. Este proceso quedaría pendiente como trabajo futuro en el caso de disponer de más tiempo.

# TO JSON

In [67]:
# convertimos nuestro dataset a formato json para pasar al equipo de Back
df_censo_json = df_cens_concat.to_json(orient='records', lines=True)

In [68]:
file_path = "..\outputs\df_censo_json.json"

with open(file_path, 'w') as file:
    file.write(df_censo_json)